In [1]:
# Install Kaggle API for dataset download
!pip install -q kaggle

# Install TensorFlow (should be pre-installed in Colab, but ensure latest version)
!pip install -q tensorflow

# Upload kaggle.json (download from Kaggle: Account > Settings > API > Create New Token)
from google.colab import files
files.upload()  # This opens a file picker; upload kaggle.json

# Set up Kaggle credentials securely
!mkdir -p ~/.kaggle  # Create .kaggle directory
!cp kaggle.json ~/.kaggle/  # Copy API token
!chmod 600 ~/.kaggle/kaggle.json  # Restrict permissions to owner only

# Optional: Mount Google Drive to save models and results
from google.colab import drive
drive.mount('/content/drive')  # Follow prompt to authenticate

Saving kaggle.json to kaggle.json
Mounted at /content/drive


In [2]:
# Download the dataset from Kaggle
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

# Unzip the dataset quietly to /content/
!unzip -q new-plant-diseases-dataset.zip

# Verify directory structure to ensure correct extraction
import os
train_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
test_dir = '/content/test/test'

# Print basic stats for report (dataset description)
print(f"Number of training classes: {len(os.listdir(train_dir))}")  # Should be 38
print(f"Training images: {sum(len(files) for _, _, files in os.walk(train_dir))}")  # ~70,295
print(f"Validation images: {sum(len(files) for _, _, files in os.walk(valid_dir))}")  # ~17,572
print(f"Test images: {len(os.listdir(test_dir))}")  # ~3,662

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 98% 2.64G/2.70G [00:29<00:01, 38.3MB/s]
100% 2.70G/2.70G [00:29<00:00, 96.8MB/s]
Number of training classes: 38
Training images: 70295
Validation images: 17572
Test images: 33


In [3]:
import os
print("Full directory structure:")
for root, dirs, files in os.walk('/content/New Plant Diseases Dataset(Augmented)/'):
    print(f"{root}: {len(dirs)} dirs, {len(files)} files")

Full directory structure:
/content/New Plant Diseases Dataset(Augmented)/: 1 dirs, 0 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented): 2 dirs, 0 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid: 38 dirs, 0 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Tomato___Tomato_Yellow_Leaf_Curl_Virus: 0 dirs, 490 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Tomato___Bacterial_spot: 0 dirs, 425 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Cherry_(including_sour)___Powdery_mildew: 0 dirs, 421 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Apple___Cedar_apple_rust: 0 dirs, 440 files
/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Grape___Esca_(Black_Measles): 0 dirs, 

In [4]:
# Import essential libraries for deep learning and visualization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications  # For model building and transfer learning
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For data loading/augmentation
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score  # For results
import seaborn as sns  # For confusion matrix visualization
import pandas as pd  # For comparison tables
import os

# Define constants for reproducibility and clarity
IMG_SIZE = (224, 224)  # Standard for transfer learning (e.g., ResNet, VGG); resizes from 256x256
BATCH_SIZE = 32  # Suitable for Colab T4 GPU (~16GB VRAM)
NUM_CLASSES = 38  # Fixed: 38 plant disease/health classes
EPOCHS = 20  # Initial; will use early stopping to prevent overfitting
SEED = 42  # For reproducible shuffling/splits

In [5]:
# Import additional library for class weight computation
from sklearn.utils.class_weight import compute_class_weight

# Define directory paths (adjusted based on structure)
train_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
# Test directory not found; will use valid for evaluation or split later if needed
test_dir = None  # Placeholder; revisit if test set is located

# Function to remove corrupted images (cleansing)
def remove_corrupted_images(directory):
    num_removed = 0
    if directory:  # Only proceed if directory exists
        for subdir, _, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(subdir, file)
                try:
                    img = tf.io.read_file(file_path)
                    tf.image.decode_jpeg(img)  # Attempt to decode
                except:
                    os.remove(file_path)
                    num_removed += 1
    print(f"Removed {num_removed} corrupted images from {directory if directory else 'no directory'}.")

# Clean train and valid sets
remove_corrupted_images(train_dir)
remove_corrupted_images(valid_dir)

# Data generators with augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=20,  # Rotate up to 20 degrees for robustness
    width_shift_range=0.2,  # Shift width by 20%
    height_shift_range=0.2,  # Shift height by 20%
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Zoom in/out by 20%
    horizontal_flip=True,  # Flip horizontally
    fill_mode='nearest',  # Fill gaps with nearest pixel
    validation_split=0.1  # Reserve 10% of train for a holdout test if needed
)

valid_datagen = ImageDataGenerator(rescale=1./255)  # Only normalize validation

# Load and preprocess datasets
train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,  # Resize to 224x224
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Multi-class with one-hot encoding
    subset='training',  # Use 90% of train
    seed=SEED
)

valid_ds = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,  # Keep order for evaluation
    seed=SEED
)

# Optional: Create a holdout test set from train (10% split)
test_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # 10% holdout
    seed=SEED,
    shuffle=False
)

# Compute class weights to handle imbalance
train_classes = train_ds.classes
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_classes),
    y=train_classes
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

# Verify dataset sizes
print(f"Train batches: {len(train_ds)}")
print(f"Validation batches: {len(valid_ds)}")
print(f"Test batches: {len(test_ds) if test_ds else 'Not created'}")

Removed 0 corrupted images from /content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train.
Removed 0 corrupted images from /content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid.
Found 63282 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Found 7013 images belonging to 38 classes.
Class weights: {0: np.float64(0.9175293605915615), 1: np.float64(0.9308640522491247), 2: np.float64(1.0513357256778308), 3: np.float64(0.9210817419655333), 4: np.float64(1.0185417672621921), 5: np.float64(1.0992183428869202), 6: np.float64(1.0129658086822897), 7: np.float64(1.1267359874652803), 8: np.float64(0.9698985378414002), 9: np.float64(0.9693339868880583), 10: np.float64(0.9948122995661196), 11: np.float64(0.9795975232198142), 12: np.float64(0.9637244152046783), 13: np.float64(1.074397283531409), 14: np.float64(1.0934443791685386), 15: np.float64(0.9205725757179017), 16: np.float64(1.0062331054221656), 17: 

In [ ]:
# Define callbacks for training optimization
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True  # Restore best model weights
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3  # Reduce LR if no improvement
    ),
    keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/plant_model_{epoch}.h5',
        save_best_only=True  # Save only the best model
    )
]

# Function to build, compile, train, and evaluate a model
def build_train_evaluate_model(base_model=None, name='Model'):
    if base_model:
        base_model.trainable = False  # Freeze base layers for transfer learning
        model = keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),  # Reduce spatial dimensions
            layers.Dense(128, activation='relu'),  # Dense layer for feature extraction
            layers.Dropout(0.5),  # Prevent overfitting
            layers.Dense(NUM_CLASSES, activation='softmax')  # Output layer for 38 classes
        ])
    else:  # Custom CNN
        model = keras.Sequential([
            layers.Conv2D(32, 3, activation='relu', input_shape=(*IMG_SIZE, 3)),  # Initial conv layer
            layers.MaxPooling2D(),  # Reduce spatial size
            layers.Conv2D(64, 3, activation='relu'),  # Deeper conv layer
            layers.MaxPooling2D(),
            layers.Conv2D(128, 3, activation='relu'),
            layers.MaxPooling2D(),
            layers.Flatten(),  # Flatten for dense layers
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(NUM_CLASSES, activation='softmax')
        ])

    model.compile(
        optimizer='adam',  # Adaptive optimizer for efficient training
        loss='categorical_crossentropy',  # Multi-class loss
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]  # Track performance
    )

    history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=valid_ds,
        class_weight=class_weights,  # Apply imbalance weights
        callbacks=callbacks
    )

    # Evaluate on validation set
    val_preds = np.argmax(model.predict(valid_ds), axis=1)
    val_true = np.argmax([y for x, y in valid_ds], axis=1)  # Extract true labels
    print(f"{name} - Classification Report:\n", classification_report(val_true, val_preds, target_names=class_names[:5]))  # Sample 5 classes
    return model, history

# Define and train models with justifications
# 1. Custom CNN (Baseline)
# Justification: Simple architecture to understand convolutions and pooling. Suitable as a baseline but may struggle with complex features due to limited depth. ReLU for non-linearity, softmax for multi-class output.
model1, history1 = build_train_evaluate_model(name='Custom CNN')

# 2. VGG16 (Transfer Learning)
# Justification: Pre-trained on ImageNet, deep with uniform 3x3 convs for fine details. Frozen base leverages learned features; ReLU and softmax for consistency.
vgg_base = applications.VGG16(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
model2, history2 = build_train_evaluate_model(vgg_base, 'VGG16')

# 3. ResNet50 (Transfer Learning)
# Justification: Uses residual connections to prevent vanishing gradients in deep nets. Pre-trained on ImageNet, ideal for complex plant disease patterns; ReLU and softmax.
resnet_base = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
model3, history3 = build_train_evaluate_model(resnet_base, 'ResNet50')

# 4. EfficientNetB0 (Transfer Learning)
# Justification: Compound scaling balances depth, width, and resolution for efficiency and accuracy. Pre-trained on ImageNet, optimized for limited compute; ReLU and softmax.
eff_base = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
model4, history4 = build_train_evaluate_model(eff_base, 'EfficientNetB0')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.1220 - loss: 3.2348 - precision: 0.6216 - recall: 0.0268

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 850s 425ms/step - accuracy: 0.1221 - loss: 3.2345 - precision: 0.6217 - recall: 0.0268 - val_accuracy: 0.5532 - val_loss: 1.4940 - val_precision: 0.7641 - val_recall: 0.3885 - learning_rate: 0.0010
Epoch 2/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step - accuracy: 0.5159 - loss: 1.5896 - precision: 0.7349 - recall: 0.3364

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 814s 412ms/step - accuracy: 0.5159 - loss: 1.5895 - precision: 0.7349 - recall: 0.3364 - val_accuracy: 0.7439 - val_loss: 0.8208 - val_precision: 0.8304 - val_recall: 0.6545 - learning_rate: 0.0010
Epoch 3/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.6810 - loss: 1.0316 - precision: 0.8043 - recall: 0.5604

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 808s 408ms/step - accuracy: 0.6810 - loss: 1.0315 - precision: 0.8043 - recall: 0.5604 - val_accuracy: 0.8227 - val_loss: 0.5624 - val_precision: 0.8790 - val_recall: 0.7674 - learning_rate: 0.0010
Epoch 4/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step - accuracy: 0.7457 - loss: 0.8151 - precision: 0.8334 - recall: 0.6602

...

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 801s 405ms/step - accuracy: 0.8325 - loss: 0.5274 - precision: 0.8803 - recall: 0.7885 - val_accuracy: 0.9103 - val_loss: 0.2716 - val_precision: 0.9328 - val_recall: 0.8905 - learning_rate: 0.0010
Epoch 9/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 805s 407ms/step - accuracy: 0.8490 - loss: 0.4803 - precision: 0.8885 - recall: 0.8121 - val_accuracy: 0.9107 - val_loss: 0.2733 - val_precision: 0.9272 - val_recall: 0.8953 - learning_rate: 0.0010
Epoch 10/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step - accuracy: 0.8551 - loss: 0.4682 - precision: 0.8932 - recall: 0.8199

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 800s 404ms/step - accuracy: 0.8551 - loss: 0.4682 - precision: 0.8932 - recall: 0.8199 - val_accuracy: 0.9143 - val_loss: 0.2667 - val_precision: 0.9308 - val_recall: 0.8990 - learning_rate: 0.0010
Epoch 11/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 802s 405ms/step - accuracy: 0.8648 - loss: 0.4344 - precision: 0.8994 - recall: 0.8329 - val_accuracy: 0.8786 - val_loss: 0.3899 - val_precision: 0.8983 - val_recall: 0.8631 - learning_rate: 0.0010
Epoch 12/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.8662 - loss: 0.4291 - precision: 0.8998 - recall: 0.8380

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 810s 410ms/step - accuracy: 0.8662 - loss: 0.4291 - precision: 0.8998 - recall: 0.8380 - val_accuracy: 0.9409 - val_loss: 0.1833 - val_precision: 0.9518 - val_recall: 0.9303 - learning_rate: 0.0010
Epoch 13/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 799s 404ms/step - accuracy: 0.8696 - loss: 0.4193 - precision: 0.9013 - recall: 0.8412 - val_accuracy: 0.9261 - val_loss: 0.2274 - val_precision: 0.9418 - val_recall: 0.9138 - learning_rate: 0.0010
Epoch 14/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 797s 403ms/step - accuracy: 0.8721 - loss: 0.4048 - precision: 0.9027 - recall: 0.8441 - val_accuracy: 0.9306 - val_loss: 0.2154 - val_precision: 0.9433 - val_recall: 0.9190 - learning_rate: 0.0010
Epoch 15/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 811s 410ms/step - accuracy: 0.8784 - loss: 0.3953 - precision: 0.9067 - recall: 0.8540 - val_accuracy: 0.9364 - val_loss: 0.2073 - val_precision: 0.9455 - val_recall: 0.9282 - learning_rate: 0.0010
Epoch 16/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 39

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 802s 405ms/step - accuracy: 0.9099 - loss: 0.2850 - precision: 0.9297 - recall: 0.8937 - val_accuracy: 0.9577 - val_loss: 0.1293 - val_precision: 0.9636 - val_recall: 0.9519 - learning_rate: 2.0000e-04
Epoch 17/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step - accuracy: 0.9213 - loss: 0.2507 - precision: 0.9374 - recall: 0.9067

1978/1978 ━━━━━━━━━━━━━━━━━━━━ 816s 413ms/step - accuracy: 0.9213 - loss: 0.2507 - precision: 0.9374 - recall: 0.9067 - val_accuracy: 0.9602 - val_loss: 0.1175 - val_precision: 0.9658 - val_recall: 0.9558 - learning_rate: 2.0000e-04
Epoch 18/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 820s 414ms/step - accuracy: 0.9236 - loss: 0.2359 - precision: 0.9391 - recall: 0.9087 - val_accuracy: 0.9536 - val_loss: 0.1453 - val_precision: 0.9593 - val_recall: 0.9480 - learning_rate: 2.0000e-04
Epoch 19/20
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 827s 418ms/step - accuracy: 0.9280 - loss: 0.2251 - precision: 0.9408 - recall: 0.9154 - val_accuracy: 0.9474 - val_loss: 0.1594 - val_precision: 0.9540 - val_recall: 0.9420 - learning_rate: 2.0000e-04
Epoch 20/20
 582/1978 ━━━━━━━━━━━━━━━━━━━━ 9:19 401ms/step - accuracy: 0.9324 - loss: 0.2141 - precision: 0.9456 - recall: 0.9204